Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [X] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [X] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [125]:
import numpy as np
from scipy.stats import randint, uniform
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge


def clean(X):
    X = X.copy()
    
    X["date_recorded"] = pd.to_datetime(X["date_recorded"])
    X["year_recorded"] = X["date_recorded"].dt.year
    X["month_recorded"] = X["date_recorded"].dt.month
    X["day_recorded"] = X["date_recorded"].dt.day
    X["years_after_construction"] = X["year_recorded"] - X["construction_year"]    
    
    unused = ["recorded_by", "id", "quantity_group", "date_recorded"]
    X = X.drop(columns=unused)
    
    X["latitude"] = X["latitude"].replace(-2e-08, np.nan)
    zero_cols = ["longitude", "latitude", "gps_height", "population", "construction_year"]
    for c in zero_cols:
        X[c] = X[c].replace(0, np.nan)
    print(X.head(5))    
    return X
  
target = "status_group"
train, val = train_test_split(train, test_size=len(test), stratify=train[target])

train = clean(train)
val = clean(val)
test = clean(test)

X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier()
)

params = {
    "simpleimputer__strategy": ["mean", "median"], 
    "randomforestclassifier__n_estimators": randint(50, 500), 
    "randomforestclassifier__max_depth": [5, 10, 15, 20, None], 
    "randomforestclassifier__max_features": uniform(0, 1), 
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=params,
    n_iter=3,
    cv=3,
    scoring="accuracy",
    verbose=10,
    return_train_score= True,
    n_jobs=-1
)

search.fit(X_train, y_train)
accuracy_score(y_val, search.predict(X_val))

       amount_tsh        funder  ...  day_recorded years_after_construction
41799         0.0  World Vision  ...             3                        1
20282         0.0            Fw  ...             2                       28
17494       500.0         Norad  ...            18                       31
44811         0.0       Swedish  ...             8                     2011
40895         0.0        Hewasa  ...            31                       20

[5 rows x 41 columns]
       amount_tsh             funder  ...  day_recorded years_after_construction
34249         0.0               Adra  ...             3                        5
46795         0.0  Ministry Of Water  ...            13                       15
40429         5.0    Germany Republi  ...             3                        7
14649      1500.0                Adb  ...            22                       11
44253         0.0               Dwsp  ...            16                     2012

[5 rows x 41 columns]
   amount_ts

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:  4.6min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  5.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  5.2min finished


0.8134141245298788

In [0]:
y_pred = search.predict(test)
submission = sample_submission.copy()
submission["status_group"] = y_pred
submission.to_csv("RLCHAL3.csv", index=False)
